# Intro

In [1]:
import vectorbtpro as vbt
import numpy as np
import pandas as pd
from numba import njit

In [2]:
data = vbt.GBMOHLCData.pull(
    symbols=['BTCUSD', 'ETHUSD', 'XRPUSD'],
    start='2020-01-01 UTC',
    end='2021-01-01 UTC',
    seed=vbt.symbol_dict(BTCUSD=42,ETHUSD=43,XRPUSD=44),
    n_ticks=1000,
)

In [3]:
data.plot(symbol="XRPUSD", plot_volume=False)

FigureWidget({
    'data': [{'close': array([ 99.10578454,  97.55439475,  97.10566598, ..., 101.02563645,
                              100.71769493,  96.40038857]),
              'decreasing': {'fillcolor': '#ee534f', 'line': {'color': '#ee534f'}},
              'high': array([102.83659362, 100.15423653,  99.44978601, ..., 101.05404125,
                             104.41297214, 101.80740837]),
              'increasing': {'fillcolor': '#26a69a', 'line': {'color': '#26a69a'}},
              'low': array([97.85033687, 96.61690816, 95.91608174, ..., 96.55794193, 99.70403907,
                            96.04255275]),
              'name': 'OHLC',
              'opacity': 0.75,
              'open': array([ 99.92491673,  99.07840324,  97.56997424, ...,  99.27134078,
                             101.03286826, 100.77119973]),
              'type': 'candlestick',
              'uid': 'c9b7f2be-3477-4c79-9305-5136f1bba9f3',
              'x': array([datetime.datetime(2020, 1, 1, 0, 0, tzinfo

## Basic Uniform/weighted ladder

Shows multiple ladders also using `vbt.Param`

Formula for weighted: `exit_fraction = (price - prev_price) / (last_price - init_price)`

In [4]:
@njit
def adjust_func_print(c):
    tp_stop = c.last_tp_info[c.col]
    print(tp_stop)
    return

In [5]:
vbt.pf_enums.tp_info_fields

[('init_idx', numpy.int64),
 ('init_price', numpy.float64),
 ('init_position', numpy.float64),
 ('stop', numpy.float64),
 ('exit_price', numpy.float64),
 ('exit_size', numpy.float64),
 ('exit_size_type', numpy.int64),
 ('exit_type', numpy.int64),
 ('order_type', numpy.int64),
 ('limit_delta', numpy.float64),
 ('delta_format', numpy.int64),
 ('ladder', numpy.int64),
 ('step', numpy.int64),
 ('step_idx', numpy.int64)]

In [6]:
entries = data.symbol_wrapper.fill(False)
entries.iloc[0, :] = True

pf = vbt.PF.from_signals(
    data,
    entries=entries,
    stop_ladder="Weighted", # Or just True
    tp_stop=vbt.Param([
        [0.05, 0.15, 0.3, 0.4, 0.5],
        [0.2,0.28,0.3],
    ], keys=["Ladder 1", "Ladder 2"]),
    adjust_func_nb=adjust_func_print,
)

(-1, nan, nan, nan, -1.0, nan, -1, -1, -1, nan, -1, -1, -1, -1)
(0, 101.90104973658623, 0.9813441594419249, 0.05, -1.0, nan, -1, 0, 0, nan, 1, 2, 0, 0)
(0, 101.90104973658623, 0.9813441594419249, 0.15, -1.0, nan, -1, 0, 0, nan, 1, 2, 1, 1)
(0, 101.90104973658623, 0.9813441594419249, 0.15, -1.0, nan, -1, 0, 0, nan, 1, 2, 1, 1)
(0, 101.90104973658623, 0.9813441594419249, 0.15, -1.0, nan, -1, 0, 0, nan, 1, 2, 1, 1)
(0, 101.90104973658623, 0.9813441594419249, 0.15, -1.0, nan, -1, 0, 0, nan, 1, 2, 1, 1)
(0, 101.90104973658623, 0.9813441594419249, 0.15, -1.0, nan, -1, 0, 0, nan, 1, 2, 1, 1)
(0, 101.90104973658623, 0.9813441594419249, 0.15, -1.0, nan, -1, 0, 0, nan, 1, 2, 1, 1)
(0, 101.90104973658623, 0.9813441594419249, 0.15, -1.0, nan, -1, 0, 0, nan, 1, 2, 1, 1)
(0, 101.90104973658623, 0.9813441594419249, 0.15, -1.0, nan, -1, 0, 0, nan, 1, 2, 1, 1)
(0, 101.90104973658623, 0.9813441594419249, 0.15, -1.0, nan, -1, 0, 0, nan, 1, 2, 1, 1)
(0, 101.90104973658623, 0.9813441594419249, 0.15, -1.0, 

In [7]:
pf.sharpe_ratio

tp_stop   symbol
Ladder 1  BTCUSD    0.233676
          ETHUSD    0.514910
          XRPUSD    0.654459
Ladder 2  BTCUSD    0.669206
          ETHUSD    1.135544
          XRPUSD    0.794705
Name: sharpe_ratio, dtype: float64

In [8]:
pf.wrapper.columns

MultiIndex([('Ladder 1', 'BTCUSD'),
            ('Ladder 1', 'ETHUSD'),
            ('Ladder 1', 'XRPUSD'),
            ('Ladder 2', 'BTCUSD'),
            ('Ladder 2', 'ETHUSD'),
            ('Ladder 2', 'XRPUSD')],
           names=['tp_stop', 'symbol'])

In [9]:
pf.trades.plot(column=("Ladder 1","BTCUSD"))

FigureWidget({
    'data': [{'close': array([101.90104974, 109.32646297, 109.91119193, ...,  73.41348051,
                               74.46668644,  74.67479889]),
              'decreasing': {'fillcolor': '#ee534f', 'line': {'color': '#ee534f'}},
              'high': array([102.74739663, 109.92330498, 111.49255804, ...,  73.94358956,
                              74.95323049,  75.53769137]),
              'increasing': {'fillcolor': '#26a69a', 'line': {'color': '#26a69a'}},
              'low': array([ 98.31017971, 102.04369432, 107.38461168, ...,  71.52372345,
                             72.22195621,  73.23125773]),
              'name': 'OHLC',
              'opacity': 0.5,
              'open': array([100.04963373, 102.04369432, 109.25261841, ...,  72.33972474,
                              73.38534997,  74.47621013]),
              'type': 'candlestick',
              'uid': '9397bfa0-65ff-4ee3-8319-5f6b792ca33f',
              'x': array([datetime.datetime(2020, 1, 1, 0, 0, t

## Adapted Ladders
Experimenting with the `AdaptUniform` and `AdaptWeighted` options.

`exit_fraction = 1 / (last_step + 1 - step)` for AdaptUniform

`exit_fraction = (price - prev_price) / (last_price - prev_price)` for AdaptWeighted

In [10]:
entries = data.symbol_wrapper.fill(False)
entries.iloc[0, :] = True

pf = vbt.PF.from_signals(
    data,
    entries=entries,
    stop_ladder="AdaptWeighted",
    tp_stop=[0.1, 0.2, 0.3, 0.4, 0.5],
    sl_stop=[0.1, 0.2, 0.27, 0.4, 0.5],
    adjust_func_nb=adjust_func_print,
)

(-1, nan, nan, nan, -1.0, nan, -1, -1, -1, nan, -1, -1, -1, -1)
(0, 101.90104973658623, 0.9813441594419249, 0.1, -1.0, nan, -1, 0, 0, nan, 1, 4, 0, 0)
(0, 101.90104973658623, 0.9813441594419249, 0.1, -1.0, nan, -1, 0, 0, nan, 1, 4, 0, 0)
(0, 101.90104973658623, 0.9813441594419249, 0.1, -1.0, nan, -1, 0, 0, nan, 1, 4, 0, 0)
(0, 101.90104973658623, 0.9813441594419249, 0.1, -1.0, nan, -1, 0, 0, nan, 1, 4, 0, 0)
(0, 101.90104973658623, 0.9813441594419249, 0.1, -1.0, nan, -1, 0, 0, nan, 1, 4, 0, 0)
(0, 101.90104973658623, 0.9813441594419249, 0.1, -1.0, nan, -1, 0, 0, nan, 1, 4, 0, 0)
(0, 101.90104973658623, 0.9813441594419249, 0.1, -1.0, nan, -1, 0, 0, nan, 1, 4, 0, 0)
(0, 101.90104973658623, 0.9813441594419249, 0.1, -1.0, nan, -1, 0, 0, nan, 1, 4, 0, 0)
(0, 101.90104973658623, 0.9813441594419249, 0.1, -1.0, nan, -1, 0, 0, nan, 1, 4, 0, 0)
(0, 101.90104973658623, 0.9813441594419249, 0.1, -1.0, nan, -1, 0, 0, nan, 1, 4, 0, 0)
(0, 101.90104973658623, 0.9813441594419249, 0.1, -1.0, nan, -1, 0,

In [11]:
pf.trades.plot(column=("XRPUSD"))

FigureWidget({
    'data': [{'close': array([ 99.10578454,  97.55439475,  97.10566598, ..., 101.02563645,
                              100.71769493,  96.40038857]),
              'decreasing': {'fillcolor': '#ee534f', 'line': {'color': '#ee534f'}},
              'high': array([102.83659362, 100.15423653,  99.44978601, ..., 101.05404125,
                             104.41297214, 101.80740837]),
              'increasing': {'fillcolor': '#26a69a', 'line': {'color': '#26a69a'}},
              'low': array([97.85033687, 96.61690816, 95.91608174, ..., 96.55794193, 99.70403907,
                            96.04255275]),
              'name': 'OHLC',
              'opacity': 0.5,
              'open': array([ 99.92491673,  99.07840324,  97.56997424, ...,  99.27134078,
                             101.03286826, 100.77119973]),
              'type': 'candlestick',
              'uid': '012125d0-01ad-4e1c-8edd-f204113e1485',
              'x': array([datetime.datetime(2020, 1, 1, 0, 0, tzinfo=

## Dynamic TP (V1)

Every `X%` move sell `(y * ladder_step)%` of position. Move SL after a certain amount of TPs hit

In [12]:
@njit
def adjust_func_nb_dynamic(
    c, 
    price_multiplier: float, 
    size_multiplier: float, 
    move_sl_step: int,
    move_sl_level: float,
):
    tp_stop = c.last_tp_info[c.col]
    sl_stop = c.last_sl_info[c.col]
    
    if np.isnan(tp_stop["exit_size"]) and tp_stop["step"] > -1:
        tp_stop["exit_size"] = size_multiplier * (1+tp_stop["step"]) * tp_stop["init_position"]
        tp_stop["exit_size_type"] = vbt.pf_enums.SizeType.Amount
        tp_stop["stop"] = price_multiplier * (1+tp_stop["step"])
        tp_stop["delta_format"] = vbt.pf_enums.DeltaFormat.Percent

        if tp_stop["step"] == move_sl_step:
            sl_stop["exit_size"] = 1
            sl_stop["exit_size_type"] = vbt.pf_enums.SizeType.Percent
            sl_stop["stop"] = sl_stop["init_price"] * (1 + move_sl_level)
            sl_stop["delta_format"] = vbt.pf_enums.DeltaFormat.Target
            
    return

In [13]:
entries = data.symbol_wrapper.fill(False)
entries.iloc[0, :] = True

pf3 = vbt.PF.from_signals(
    data,
    entries=entries,
    stop_ladder="Dynamic",
    adjust_func_nb=adjust_func_nb_dynamic,
    adjust_args=(
        0.05, # price multiplier
        0.08, # size_multiplier
        2, # move SL step
        0.0, # move SL level
    ),
)

In [14]:
pf3.trades.plot(column=("XRPUSD"))

FigureWidget({
    'data': [{'close': array([ 99.10578454,  97.55439475,  97.10566598, ..., 101.02563645,
                              100.71769493,  96.40038857]),
              'decreasing': {'fillcolor': '#ee534f', 'line': {'color': '#ee534f'}},
              'high': array([102.83659362, 100.15423653,  99.44978601, ..., 101.05404125,
                             104.41297214, 101.80740837]),
              'increasing': {'fillcolor': '#26a69a', 'line': {'color': '#26a69a'}},
              'low': array([97.85033687, 96.61690816, 95.91608174, ..., 96.55794193, 99.70403907,
                            96.04255275]),
              'name': 'OHLC',
              'opacity': 0.5,
              'open': array([ 99.92491673,  99.07840324,  97.56997424, ...,  99.27134078,
                             101.03286826, 100.77119973]),
              'type': 'candlestick',
              'uid': 'c257744f-2411-4443-a4ed-d8456e63c820',
              'x': array([datetime.datetime(2020, 1, 1, 0, 0, tzinfo=

## Dynamic TP (V2)

Parameter Optimization

In [15]:
@njit
def adjust_func_nb_dynamic_2(
    c, 
    price_multiplier: float, 
    size_multiplier: float, 
    move_sl_step: int,
    move_sl_level: float,
):
    tp_stop = c.last_tp_info[c.col]
    sl_stop = c.last_sl_info[c.col]

    _price_multiplier = vbt.pf_nb.select_nb(c, price_multiplier)
    _size_multiplier = vbt.pf_nb.select_nb(c, size_multiplier)
    _move_sl_step = vbt.pf_nb.select_nb(c, move_sl_step)
    _move_sl_level = vbt.pf_nb.select_nb(c, move_sl_level)
    
    if np.isnan(tp_stop["exit_size"]) and tp_stop["step"] > -1:
        tp_stop["exit_size"] = _size_multiplier * (1+tp_stop["step"]) * tp_stop["init_position"]
        tp_stop["exit_size_type"] = vbt.pf_enums.SizeType.Amount
        tp_stop["stop"] = _price_multiplier * (1+tp_stop["step"])
        tp_stop["delta_format"] = vbt.pf_enums.DeltaFormat.Percent

        if tp_stop["step"] == _move_sl_step:
            sl_stop["exit_size"] = 1
            sl_stop["exit_size_type"] = vbt.pf_enums.SizeType.Percent
            sl_stop["stop"] = sl_stop["init_price"] * (1 + _move_sl_level)
            sl_stop["delta_format"] = vbt.pf_enums.DeltaFormat.Target
            
    return

In [16]:
entries = data.symbol_wrapper.fill(False)
entries.iloc[0, :] = True

pf4 = vbt.PF.from_signals(
    data,
    entries=entries,
    stop_ladder="Dynamic",
    broadcast_named_args=dict(
        price_multiplier = vbt.Param([0.05,0.1,0.17]),
        size_multiplier = vbt.Param([0.05,0.1,0.2,0.3]),
        move_sl_step = vbt.Param([1,2,3,4]),
        move_sl_level = vbt.Param([-0.05, 0, 0.1]),
    ),
    adjust_func_nb=adjust_func_nb_dynamic_2,
    adjust_args=(
        vbt.Rep("price_multiplier"), # price multiplier
        vbt.Rep("size_multiplier"), # size_multiplier
        vbt.Rep("move_sl_step"), # move SL step
        vbt.Rep("move_sl_level"), # move SL level
    ),
)

In [17]:
pf4.total_return.sort_values(ascending=False)

price_multiplier  size_multiplier  move_sl_step  move_sl_level  symbol
0.10              0.10             4             -0.05          ETHUSD    0.300000
                                                  0.00          ETHUSD    0.300000
                                                  0.10          ETHUSD    0.300000
                                   3              0.00          ETHUSD    0.300000
                                   2              0.00          ETHUSD    0.300000
                                                                            ...   
0.17              0.05             4              0.10          ETHUSD   -0.470052
                                   3              0.10          ETHUSD   -0.470052
                                                  0.00          ETHUSD   -0.470052
                                   4              0.00          ETHUSD   -0.470052
                                                 -0.05          ETHUSD   -0.470052
Name: total_retu

In [18]:
pf4.trades.plot(column=(0.1,0.1,4,-0.05,"ETHUSD"))

FigureWidget({
    'data': [{'close': array([100.29880815, 104.07360874, 106.82572119, ...,  40.17831645,
                               38.49755271,  39.81835832]),
              'decreasing': {'fillcolor': '#ee534f', 'line': {'color': '#ee534f'}},
              'high': array([102.24206661, 106.173295  , 107.25038015, ...,  42.49746049,
                              40.2909733 ,  40.16877401]),
              'increasing': {'fillcolor': '#26a69a', 'line': {'color': '#26a69a'}},
              'low': array([ 99.14674941, 100.37893948, 103.23195193, ...,  39.59360634,
                             38.33719749,  37.99883288]),
              'name': 'OHLC',
              'opacity': 0.5,
              'open': array([100.02569329, 100.37893948, 104.01084814, ...,  41.87831862,
                              40.11885602,  38.56944046]),
              'type': 'candlestick',
              'uid': '53a11461-72e0-4c25-8598-55abdebcec0d',
              'x': array([datetime.datetime(2020, 1, 1, 0, 0, t